### Submitted by:
- Shahar Asher - 209305408
- Hadar Liel Harush - 211721568

In [188]:
import numpy as np
import re

np.random.seed(0)

### Question 1
Generate a random logical expression in the format described in section A.

In [189]:
# q1

def GenRundomRule()->str:
    """
    Generate a random rule.

    Returns:
        str: Randomly generated rule.
    """
    operators = ['and', 'or']
    pixels = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']
    rule = ''
    closing = []

    rule_len = np.random.randint(1, 10)
    for i in range(rule_len):
        is_closing = False
        if i != 0:
            rule += ' '
            rule += f'{np.random.choice(operators, 1)[0]} '
            is_closing = np.random.randint(10) != 0
        if is_closing and len(closing) == 0 and i != rule_len-1:
            rule += '('
            closing.append(')')
        else:
            is_closing = False

        if np.random.randint(10) == 0:
            rule += 'not '
        rule += f'{np.random.choice(pixels,1)[0]}'
        if not is_closing and len(closing) != 0:
            rule += ')'
            closing.pop()
    return rule

### Question 2
Check if the generated rule matches the training set presented at the beginning of the question, and if so, determine that the rule is valid.

In [190]:
treining_set_T = [[[1, 0, 0], [1, 0, 1], [0, 1, 0]],
                  [[1, 0, 0], [0, 0, 1], [1, 0, 1]],
                  [[1, 0, 0], [0, 0, 1], [0, 0, 0]]]
treining_set_F = [[[0, 0, 1], [0, 1, 0], [1, 0, 0]],
                  [[0, 1, 0], [1, 0, 1], [0, 1, 0]],
                  [[0, 1, 1], [1, 1, 0], [0, 1, 1]]]

In [191]:
def removeP(lst):
    """
    Remove 'p' identifiers from the list.

    Args:
        lst (list): List containing logical components.

    Returns:
        list: List with 'p' identifiers removed.
    """
    generare_rule_list = [re.findall(r'\(||\)||not||[0-9]||or||and', rule) for rule in lst.split(' ')]
    generare_rule_list = [r for rule in generare_rule_list for r in rule if r != '']
    return generare_rule_list

In [192]:
def addClosing(lst):
    """
    Add closing parentheses to the list of logical components.

    Args:
        lst (list): List containing logical components.

    Returns:
        list: List with closing parentheses added.
    """
    is_close = True
    is_in = False

    list_len = len(lst)
    i = 0
    while i <= list_len:
        if is_close and not is_in:
            lst.insert(i, '(')
            is_close = False
            list_len+=1
        elif (i == list_len or (lst[i] == 'and' and not is_in)) and not is_close:
                lst.insert(i, ')')
                is_close = True
                list_len+=1
                i+=1
        elif lst[i] == '(':
            is_in = True
        elif lst[i] == ')':
            is_in = False
        i+=1
    return lst

In [193]:
def removeClosing(lst, rule_list_len, rule):
    """
    Remove closing parentheses and associated components from the list.

    Args:
        lst (list): List containing logical components.
        rule (int): Index of the rule.

    Returns:
        tuple: Tuple containing the modified list and its length.
    """
    temp_lst = []
    i = rule + 1
    lst[rule] = temp_lst
    while lst[i] != ')':
        if lst[i] == '(':
            lst, rule_list_len = removeClosing(lst, rule_list_len, i)
        elif lst[i] == 'not':
            lst[i+1] = f'-{lst[i+1]}'
            lst.pop(i)
            rule_list_len -= 1
        temp_lst.append(lst[i])
        lst.pop(i)
        rule_list_len -= 1
    else:
        lst.pop(i)
        rule_list_len -= 1
    return (lst, rule_list_len)

In [194]:
def removeNotAndClosing(lst):
    """
    Remove negations and closing parentheses from the list of logical components.

    Args:
        lst (list): List containing logical components.

    Returns:
        list: List with negations and closing parentheses removed.
    """
    rule_list_len = len(lst)
    rule = 0
    while rule < rule_list_len:
        if lst[rule] == 'not':
            lst[rule+1] = f'-{lst[rule+1]}'
            lst.pop(rule)
            rule_list_len -= 1
        elif lst[rule] == '(':
            lst, rule_list_len = removeClosing(lst, rule_list_len, rule)
        rule+=1
    return lst

In [195]:
def to_int(lst):
    """
    Convert elements of the list to integers.

    Args:
        lst (list): List containing elements to be converted to integers.

    Returns:
        list: List with elements converted to integers.
    """
    filtered_list = []
    for rule in lst:
        if isinstance(rule, list):
            filtered_list.append(to_int(rule))
        elif rule != 'or' and rule != 'and':
            filtered_list.append(int(rule))
        else:
            filtered_list.append(rule)
    return filtered_list

In [196]:
def checkBoolRule(rule_lst):
    """
    Check the boolean rule represented by the list of logical components.

    Args:
        rule_lst (list): List containing logical components of the rule.

    Returns:
        bool: True if the rule is satisfied, False otherwise.
    """
    bool_rule_lst = []
    operator = ''
    for rule in range(len(rule_lst)):
        if isinstance(rule_lst[rule], list):
            bool_lst_temp = checkBoolRule(rule_lst[rule])
            if bool_rule_lst == []:
                bool_rule_lst.append(bool_lst_temp)
            elif operator == 'or':
                bool_rule_lst[0] = bool_rule_lst[0] or bool_lst_temp
            elif operator == 'and':
                bool_rule_lst[0] = bool_rule_lst[0] and bool_lst_temp
        elif bool_rule_lst == []:
            bool_rule_lst.append(rule_lst[rule])
        elif rule_lst[rule] == 'or':
            operator = 'or'
        elif rule_lst[rule] == 'and':
            operator = 'and'
        else:
            if operator == 'or':
                bool_rule_lst[0] = bool_rule_lst[0] or rule_lst[rule]
            elif operator == 'and':
                bool_rule_lst[0] = bool_rule_lst[0] and rule_lst[rule]
    return bool_rule_lst[0]

In [197]:
def checkForSet(st, rule_lst):
    """
    Check if the rule list is satisfied by the training set.

    Args:
        st (list): Training set.
        rule_lst (list): List representing the rule.

    Returns:
        bool: True if the rule is satisfied, False otherwise.
    """
    check_rule_lst = []
    for rule in rule_lst:
        if isinstance(rule, list):
            check_rule_lst.append(checkForSet(st, rule))
        elif rule == 'or' or rule == 'and':
            check_rule_lst.append(rule)
        else:
            row = (np.abs(rule)//3)-1 if np.abs(rule)%3 == 0 else np.abs(rule)//3
            if rule > 0:
                check_rule_lst.append(st[row][(np.abs(rule)%3)-1] != 0)
            else:
                check_rule_lst.append(st[row][(np.abs(rule)%3)-1] == 0)
    return checkBoolRule(check_rule_lst)

In [198]:
def check_rule(rule_lst, treining_set):
    """
    Check if the given rule list is satisfied by the training set.

    Args:
        rule_list (list): List representing the rule.
        training_set (list): Training set.

    Returns:
        list: List of boolean values indicating whether the rule is satisfied for each set in the training set.
    """
    check_rule_lst = []
    for st in treining_set:
        check_rule_lst.append(checkForSet(st, rule_lst))
    return check_rule_lst

In [199]:
# q2

def legalRule():
    """
    Generate a legal rule.

    Returns:
        tuple: A tuple containing the generated rule and its classification.
    """
    while True:
        generare_rule = GenRundomRule()
        generare_rule_list = removeP(generare_rule)
        generare_rule_list = addClosing(generare_rule_list)
        generare_rule_list = removeNotAndClosing(generare_rule_list)
        generare_rule_list = to_int(generare_rule_list)
        check_rule_T = check_rule(generare_rule_list, treining_set_T)
        check_rule_F = check_rule(generare_rule_list, treining_set_F)

        if True in check_rule_T and False not in check_rule_T:
            if True not in check_rule_F:
                return (generare_rule, True)
        elif True in check_rule_F and False not in check_rule_F:
            if True not in check_rule_T:
                return (generare_rule, False)

In [200]:
generare_rule, classification = legalRule()
print(f'legal rule as --{classification}--:\n--------------------\n{generare_rule}')


legal rule as --False--:
--------------------
p3 or (p4 and p2)


### Question 3
Apply the valid rule to the example.

In [201]:
training_example = [[[1, 0, 0], [0, 1, 0], [0, 0, 1]]]

In [202]:
# q3

def activate_rule(rule, trining_set):
    """
    Activate a rule on the training set.

    Args:
        rule (str): Rule to activate.
        training_set (list): Training set.

    Returns:
        bool: True if the rule is activated, False otherwise.
    """
    generare_rule_list = removeP(rule)
    generare_rule_list = addClosing(generare_rule_list)
    generare_rule_list = removeNotAndClosing(generare_rule_list)
    generare_rule_list = to_int(generare_rule_list)
    check_rule_ex = check_rule(generare_rule_list, trining_set)

    if True in check_rule_ex and False not in check_rule_ex:
        return True
    else:
        return False

In [203]:
if activate_rule(generare_rule, training_example):
    print(f'The rule is activated on the training example.\nand it gives: --- True ---')
else:
    print(f'The rule is not activated on the training example.\nand it gives: --- False ---')

The rule is not activated on the training example.
and it gives: --- False ---


### Question 4
Use a loop that utilizes the previous parts of the code to find 20 rules that yield True on the example, and 20 rules that yield False on the same example.

In [204]:
# q4

def find_rule(bool_r):
    """
    Find legal rules that satisfy the conditions.

    Args:
        bool_r (bool): Boolean value indicating the condition.
        num_rules (int): Number of rules to find.

    Returns:
        list: List of found rules.
    """
    lst = []
    for i in range(20):
        while True:
            temp_rule = legalRule()
            if temp_rule[1] == bool_r and activate_rule(temp_rule[0], training_example) and temp_rule not in lst:
                lst.append(temp_rule)
                break
    return lst

In [205]:
rules_lst_T = find_rule(True)
rules_lst_F = find_rule(False)

print(rules_lst_T)
print(rules_lst_F)

[('p1 and (p5 or p1) or p7', True), ('p1 or (not p6 and not p5)', True), ('p1', True), ('p1 and (p5 and p8) or (p6 or p1) or (p8 and not p8)', True), ('p1 and (p8 or p1)', True), ('p1 and (p1 or p9)', True), ('p1 and (not p7 or p9) or (p9 and p3) or (p2 or p5)', True), ('not p4 or p5 or (p8 and p6) and p1', True), ('p1 and p1', True), ('p4 or (p3 and p9) or (p1 or p9) or (p4 and p8) and p1', True), ('p1 or (p7 and p1) and (p9 or p1) or (p3 or p6) or p6', True), ('p1 or p1', True), ('p1 and (p3 or p8) or not p2', True), ('not p2 or (p4 and p1) and p1', True), ('p1 or (p6 and p9)', True), ('p1 and (p2 and p2) or (p1 or p7)', True), ('p1 and (p4 or p1) or (p9 or p7) or not p9', True), ('p1 and (p1 or p6) or (p5 and p5) or (p3 and p9) or p6', True), ('p1 or (p8 and p7) and (p6 or p1) or (p1 and not p7)', True), ('p1 and (p7 and p2) or (p1 or p6)', True)]
[('p2 or (p5 or p3) and (not p4 or p1) or not p7', False), ('not p1 or p2 or (p5 and p9)', False), ('p3 or (p3 or p3) or not p1 or (p1 an

### Question 5
Write the results to a text file according to the following guidelines:
- The file name will be txt.rules.
- The first line in the file will be:  
  `# Rules that give True on test`
- Following that, the 20 rules that yield true on the test will appear, with each rule written on a separate line.
- The 22nd line in the file will be:  
  `# Rules that give False on test`
- Following that, the 20 rules that yield false on the test will appear, with each rule written on a separate line.
- There should be no empty lines in the file.

In [206]:
# q5

with open('.//rules.txt', 'w') as file:
    file.write('# Rules that give True on test\n')
    for rule in rules_lst_T:
        file.write(f'{rule[0]}\n')
    file.write('# Rules that give False on test\n')
    for rule in rules_lst_F:
        file.write(f'{rule[0]}\n')
with open('.//rules.txt', 'r+') as file:
    content = file.read()
    content = content.rstrip('\n')
with open('.//rules.txt', 'w') as file:
    file.write(content)